## pdftree
根节点为virtualNode，根据headline及其level构建pdftree。
没有headline的默认为根节点的子节点

In [4]:
from pdfextractor import extractor
import pdfextractor
import pickle
import jieba
import pandas as pd
import re

In [ ]:
d = pd.read_csv('/home/zhuyuhe/mydata/jmd/percent/jmd_text_map.txt', header = None, sep = '\t', encoding = 'utf-8')
d.columns = ['index','文本', '意图', 'pdfurl', 'pdf文本']
d = d[d['文本'].apply(lambda x: x.__len__() > 10)]
d = d[d['文本'].duplicated() == False]
d.to_csv('/home/zhuyuhe/mydata/jmd/percent/jmd_text_map.txt', header = None,index=None, sep = '\t', encoding = 'utf-8')

In [ ]:
a = "创显将大力发展软硬件结合的教学解决方案，配套以不同层次的专业培训服务"
pdflist = ['2017年，公司将大力发展软硬件结合的视频教学解决方案，形成精品录播室，标准录播，配套以不同的专业培训服务', '2017年，公司将在既有的教师化信息能力平台业务基础上，进一步深化细化培养服务']
print(isMatch(a, pdflist, 0.6))

In [6]:
pdfurl = ''
savePath = '/home/zhuyuhe/mydata/jmd/percent/'
# fileObject = open(savePath + 'jmd_pdf_map.txt', 'w') 
def write_unicode(text, charset='utf-8'):
    return text.encode(charset)
def MatchTwoText(t1, t2, threshold = 0.5):
    """
    检查两个句子是否匹配
    """
#     print t1
    ct1 = ' '.join(jieba.cut(t1)).split()
    ct2 = ' '.join(jieba.cut(t2)).split()
    commonL = set(ct1).intersection(set(ct2)).__len__()
#     print commonL
#     print ct1
#     print set(ct1).__len__()
    if commonL * 1.0 / set(ct1).__len__() > threshold:
#         print("match " + str(commonL * 1.0/ set(ct1).__len__() * 100) + '% ')
#         print("文章句子:" + '\n' + t1)
#         print("PDF句子: " + '\n' + t2)
#         print('\n')
#         fileObject.write(t1.encode('utf-8'))
#         fileObject.write('\t')
#         fileObject.write(t2.encode('utf-8'))
#         fileObject.write('\t')
#         fileObject.write(pdfurl)
#         fileObject.write('\n')
        return True
    return False
    
def isMatch(text, pdfTextList, threshold = 0.5):
    """
    @parameter text: 待检查句子
    @parameter pdfTextList: PDF句子与表格信息列表
    @parameter threshold: 相似度为多少时返回true
    @return： boolean值，pdf列表中是否有与待检查句子非常相似的句子
    """
    for pdftext in pdfTextList:
        if MatchTwoText(text, pdftext, threshold):
            return True
    return False

def get_node_texts(pdf_nodes):
    '''Given a list of nodes, get the texts of these nodes
    '''
    texts = []
    for node in pdf_nodes:
        text = node.obj.get_text()
        if len(text) <= 25:
            continue
        if u'□' in text or u'√' in text:
            continue
        texts.append(text)
    return texts

#percent: 取pdf的前多少页
def extractPDF(pdfPath, percent, source = ['table', 'text', 'para']):
    savePath = '/home/zhuyuhe/mydata/jmd/percent/'
    ex = extractor.StockFinanceExtractor(pdfPath, percent)

    nodes = ex.get_pdf_objects(filter_headline=True)

    paraList = []
    textList = []
    textNodes = []
    tableList = []
    tableNodes = []
    senList = []
    for node in nodes:
        if isinstance(node.obj, pdfextractor.pdftree.VirtualNode):
            continue
        if isinstance(node.obj, pdfextractor.page.Table):
            tableNodes.append(node)
        elif  not isinstance(node.obj, pdfextractor.page.Catalog):
            textNodes.append(node)
    if 'table' in source:
        tableList = get_node_texts(tableNodes)
        tableList = [t.replace('|', '').split('\n') for t in tableList]
        #二维数组转化为一维
        tableList = sum(tableList,[])
    textList = get_node_texts(textNodes)
    if 'para' in source:
        paraList = [t.replace('\n', '') for t in textList]
        paraList = filter(lambda x: x != '', paraList)
    elif 'text' in source:
        senList = sum([t.encode('utf-8').split('。') for t in textList],[])
        senList = [t.replace('\n', '') for t in senList]
        senList = filter(lambda x: x != '', senList)

    pdf = []
#     pdf.append(textList)
#     pdf.append(tableList)
#     pdf = sum(pdf, [])
    pdf.extend(senList)
    pdf.extend(tableList)
    pdf.extend(paraList)
    print pdf.__len__()
    
    return pdf


#     
#     fileObject = open(savePath + 'cxkjTextList.txt', 'w')  
#     for text in textList:  
#         fileObject.write(text)  
#         fileObject.write('\n')
#     fileObject.close()
#     fileObject2 = open(savePath + 'cxkjTableList.txt', 'w')  
#     for table in tableList:  
#         fileObject2.write(write_unicode(table))  
#         fileObject2.write('\n')  
#     fileObject2.close()

In [8]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

def splitByPeriod(P):
    """
    split paragraph to sentences
    """
    senList = sum([t.encode('utf-8').split('。') for t in P],[])
    senList = [t.replace('\n', '') for t in senList]
    senList = filter(lambda x: x != '', senList)
    return senList

savePath = '/home/zhuyuhe/mydata/jmd/percent/'
fp = open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'rb')
jmdata_mapping = pickle.load(fp)

content_url_map = {}
for d in jmdata_mapping[2:]:
    content_url_map[d['pdf_url']] = d['content']

urls = content_url_map.keys()

num = 1
result = {}
jmdata_map = {}

for path in urls:
    print "Article " + str(num) + ": " 
    num += 1
    
    print path
    pdfurl = path
    jmdata_map[pdfurl] = {}
    
    cxkj = content_url_map[path]
    #print cxkj
    cxkj = cxkj.strip().encode('utf-8').split('。')
    cxkj = [x.replace('\n','') for x in cxkj]
    
    try :
        pdflist = extractPDF(path, 0.5, ['para'])
    except BaseException:
        print 'error extractPDF'
        continue
    
    print "------"
    jmdata_map[pdfurl]['content'] = cxkj
    jmdata_map[pdfurl]['para'] = pdflist
    jmdata_map[pdfurl]['sen'] = splitByPeriod(pdflist)
    jmdata_map[pdfurl]['mapcontent'] = []
    
#     fileObject = open(savePath + 'cxkjTextList.txt')
#     textlist = fileObject.read().split('\n')
#     fileObject = open(savePath + 'cxkjTableList.txt')
#     tablelist = fileObject.read().split('\n')
#     fileObject.close()

#     PDFTextList = []
#     PDFTextList.append(textlist)
#     PDFTextList.append(tablelist)
#     PDFTextList = sum(PDFTextList, [])
#     print PDFTextList.__len__()
    print('**************************')
    count = 0
    for text in cxkj:
        if not (len(text) == 0):
            if isMatch(text, pdflist, 0.6):
                jmdata_map[pdfurl]['mapcontent'].append(text)
                count+=1
                
    print str(count * 1.0/ cxkj.__len__() * 100) + '% 的句子来自PDF'
    result[path] = count * 1.0/ cxkj.__len__() * 100
    print('***************************')
    
# fileObject.close()


Article 1: 
http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 20s


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


Construct Pdf Tree time: 0s
124
------
**************************


Loading model cost 0.351 seconds.
Prefix dict has been built succesfully.


75.0% 的句子来自PDF
***************************
Article 2: 
http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF
Download pdf time: 3s
download http status code: 200
Pdfminer Parsed time: 51s
Construct Pdf Tree time: 0s
145
------
**************************
66.6666666667% 的句子来自PDF
***************************
Article 3: 
http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-04-15/601801_2016_n.pdf
Download pdf time: 0s
download http status code: 200
Pdfminer Parsed time: 40s
Construct Pdf Tree time: 0s
244
------
**************************
58.3333333333% 的句子来自PDF
***************************
Article 4: 
http://disclosure.szse.cn/finalpage/2016-10-25/1202780794.PDF
Download pdf time: 3s
download http status code: 200
Pdfminer Parsed time: 4s
Construct Pdf Tree time: 0s
36
------
**************************
64.2857142857% 的句子来自PDF
***************************
Article 5: 
http://disclosure.szse.cn/finalpage/2016-10-18/1202763826.PDF
Download pdf time: 1s
download http status 

In [11]:
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['content'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['mapcontent'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['para'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['sen'])
jmdata_map = dict(filter(lambda x: len(x[1]) != 0, jmdata_map.items()))

18
12
145
255


In [12]:
with open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map.pkl', 'wb') as pickle_file:
     pickle.dump(jmdata_map, pickle_file , protocol=2)

In [ ]:
pdftext = dataJoin[dataJoin['pdfurl'] == 'http://disclosure.szse.cn/finalpage/2016-10-27/1202793355.PDF'][dataJoin['pdf文本'] != '']['pdf文本']
pdftext.to_csv('/home/zhuyuhe/mydata/jmd/predict/pdftext.txt', header = None, index=None, sep = '\t', encoding = 'utf-8')

In [ ]:
#计算pdf意图符合文章意图的句子比例
pdftext = dataJoin[dataJoin['pdfurl'] == 'http://disclosure.szse.cn/finalpage/2016-10-27/1202793355.PDF'][dataJoin['pdf文本'] != '']
pdftext_intent = pd.read_csv('/home/zhuyuhe/mydata/jmd/predict/cxkjpdftext_predict.txt', header = None)
pdftext['pdfintent'] = pdftext_intent.values
pdftext['pdfintent'] = pdftext['pdfintent'].apply(lambda x: x.replace('__label__', ''))
pdftext['意图'] = pdftext['意图'].apply(lambda x: '|'.join(x.split('|')[:2]))
bol = pdftext['意图'] == pdftext['pdfintent']
print "文本句子有 " + str(len(bol)) + " 句"
print "pdf意图符合文章意图的句子比例为" + str(bol.value_counts()[1] * 1.0 / len(bol) * 100) + "%"

In [ ]:
print qtjypdftext[43]

In [ ]:
import numpy as np
result = np.array(result.values())
result[result != 0]
#0%的原因： 1.科大讯飞  -  表格描述；上一季度财报描述  ×2 
#           2.新南洋    -  财报中大部分均为表格，并且文章使用了其他季度的财报
#           3.中国高科  -  前两句话描述财报表格，
#           4.洪涛股份  -  文章与财报不匹配，2015年年报 = 2016年第一季度财报？

In [ ]:
text_nodes = []

In [ ]:
for n in nodes:
    if isinstance(n.obj, pdfextractor.pdftree.VirtualNode):
        continue
    if len(n.obj.get_text()) <= 25:
        continue
    else:
        text = n.obj.get_text()
        if u'□' in text or u'√' in text:
            continue
        text_nodes.append(n)
        print n

In [ ]:
node= text_nodes[10]

In [ ]:
isinstance(node.obj,pdfextractor.paragraph.Paragraph)

In [ ]:
def get_child_headlines(pdf_node):
    '''Get headlines of children of a node
    '''
    headlines = []
    for node in pdf_node.children:
        if isinstance(node.obj, pdfextractor.paragraph.Headline):
            headlines.append(node)
    return headlines

In [ ]:
def get_child_paragraphs(pdf_node):
    '''Get paragraphs of children of a node
    '''
    paragraphs = []
    for node in pdf_node.children:
        if isinstance(node.obj, pdfextractor.paragraph.Paragraph):
            paragraphs.append(node)
    return paragraphs

In [ ]:
def get_paragraph_headlines(pdf_node):
    '''Get headlines of a paragraph
    Input:
        - pdf_node: a pdf node
    Output:
        - headlines: a list of sorted headline nodes
        - texts: a list of sorted headline texts
    '''
    headlines = []
    node = pdf_node.parent
    while not isinstance(node.obj,pdfextractor.pdftree.VirtualNode):
        headlines.append(node)
        node = node.parent
    headlines = list(reversed(headlines))
    return headlines, get_node_texts(headlines)

In [ ]:
print get_paragraph_headlines(text_nodes[15])[1][-1]

## 预测

In [ ]:
import requests

In [ ]:
host = 'https://alpha-surreal.aidigger.com/api/v1/classification/intent/finance/0'

In [ ]:
hd_intents = []
all_intents = []
for node in text_nodes:
    content = node.obj.get_text()
    headline = get_paragraph_headlines(node)[1][-1]
    intents = requests.post(host,json={"query":content}).json()
    head_intents = requests.post(host,json={"query":headline}).json()
    for i,item in enumerate(head_intents['predict']):
        intents['predict'][i]['prob'] += item['prob']
    all_intents.append(intents)
    intent = max(intents['predict'],key=lambda item:item['prob'])
    hd_intents.append(intent)

In [ ]:
def print_intent(it):
    for i,item in enumerate(it):
        print i,item['name'],item['prob']

In [ ]:
print_intent(all_intents[0]['predict'])

In [ ]:
for i,intent in enumerate(hd_intents):
    print i
    print text_nodes[i]
    print ' ==> '.join(get_paragraph_headlines(text_nodes[i])[1])
    print intent['name'],'\n'


# 根据芥末堆提供的意图模板，得到意图推荐的准确率和召回率

In [ ]:
query = requests.post("https://surreal.aidigger.com/api/v1/classification/intent/finance/0",json={ 
    "query": "公司立足校园、面向家庭提供教育信息服务，深耕教育十余年已形成覆盖校内外多场景、涵盖教育主管部门、学校、教师、家长及学生等 B 端、C 端主体的业务服务体系。公司针对不同用户群体多场景、多元化需求，持续优化服务供给，逐步构建了家校互动升级业务、EdSaaS 业务、学科升学业务及继续教育业务，报告期内公司主营业务未发生变化。"
}).json()

In [ ]:
def hasCN(s):
    """
    判断一段文本是否有中文
    """
    for ch in s.decode('utf-8'):
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False



In [ ]:
dataJoin = pd.read_csv('/home/zhuyuhe/mydata/jmd/percent/dataJoin.txt', delimiter='\t', header = None)

dataJoin.columns = ['文本', '意图', 'pdfurl']

dataJoin = dataJoin[dataJoin['文本'].apply(lambda x: len(x) > 30)]
dataJoin = dataJoin[dataJoin['文本'].duplicated() == False]
dataJoin = dataJoin[dataJoin['文本'].apply(lambda x: hasCN(x))]

dataJoin.index = range(len(dataJoin))
dataJoin['意图'] = dataJoin['意图'].apply(lambda x: intentMap(x))

dataJoin = dataJoin[dataJoin['意图'].apply(lambda x: x!= 'Other')]

In [ ]:
for i in dataJoin['意图'].unique():
    print i

In [ ]:
a = dataJoin[dataJoin.pdfurl == 'http://disclosure.szse.cn/finalpage/2016-10-27/1202791188.PDF']
b = dataJoin[dataJoin.pdfurl == 'http://disclosure.szse.cn/finalpage/2016-10-25/1202780960.PDF'] 
c = dataJoin[dataJoin.pdfurl == 'http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF']
print a.loc[119, '文本']

In [ ]:
len(dataJoin)

In [ ]:
print a['意图'].value_counts()
print b['意图'].value_counts()
print c['意图'].value_counts()

In [36]:
import operator
d = {'a':1, 'b':2, 'c':3}
sorted(d.items(), key=operator.itemgetter(1))

[('a', 1), ('b', 2), ('c', 3)]

In [38]:
d.keys()[d.values().index(max(d.values()))]

'c'